<a href="https://colab.research.google.com/github/ChaelinL/Deeplearning_for_everyone/blob/main/19%EC%9E%A5_%EC%8B%A4%EC%8A%B5(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN 모델 만들기

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 생성자 모델 만들기
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))  # 이때, 128은 임의의 수. 7*7 -> 두 번의 UpSampling2D을 통해 28*28로 변하게 됨
generator.add(BatchNormalization())  # 배치 정규화를 통해 다음 층으로 입력될 값을 일정하게 재배치 (정규분포화)
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())  # 이미지의 가로세로 크기를 2배씩 늘려줌 (따라서 MNIST 손글씨의 크기가 28*28인데 7*7로 설정. -> 7 14 28)
generator.add(Conv2D(64, kernel_size=5, padding='same'))  # 판별자와 비교할 '진짜'와 똑같은 크기로 만들어주기 위한 패딩 설정
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))  # LeakyReLU는 ReLU함수에서 x값이 음수면 무조건 0이 되는 기울기 소실문제 보완한 것으로, 0보다 작을 경우 0.2 곱하라는 의미
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [3]:
# 판별자 모델 만들기
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))  # strides-> 커널 윈도를 몇 칸씩 이동시킬지 설정(차원을 줄여 새로운 특징 뽑아주는 효과)
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))  # 판별의 결과가 진짜 혹은 가짜 둘 중 하나임으로 sigmoid 함수 사용
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False  # 판별자 자신이 학습되지 않게끔 학습 기능 꺼줌

In [4]:
# 생성자와 판별자 모델을 연결시키는 GAN 모델 만들기
ginput = Input(shape=(100,))  # 랜덤한 100개의 벡터를 케라스의 Input()함수에 넣어 생성자에 입력할 ginput 만드는 과정
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         865281    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [5]:
# 신경망을 실행시키는 함수 만들기
def gan_train(epoch, batch_size, saving_interval):
  
  # MNIST 데이터 불러오기 (테스트 과정 없이 이미지만 사용할 것이므로 X_train만 호출)
  (X_train, _), (_, _) = mnist.load_data()

  # 가로 28픽셀, 세로 28픽셀, 흑백이므로 1을 설정
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

  # 0~255 사이 픽셀 값에서 127.5를 뺀 후 127.5로 나누면 -1~1 사이의 값으로 바뀐다
  X_train = (X_train-127.5)/127.5

  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch):
    # 실제 데이터를 판별자에 입력하는 과정
    idx = np.random.randint(0, X_train.shape[0], batch_size)  # 0부터 X_train 개수 사이의 숫자를 랜덤하게 선택해 batch_size만큼 반복해서 가져옴
    imgs = X_train[idx]  # 선택된 이미지 불러오기
    d_loss_real = discriminator.train_on_batch(imgs, true)

    # 가상 이미지를 판별자에 입력하는 과정
    noise = np.random.normal(0, 1, (batch_size, 100))  # 0부터 1까지의 실수 중 batch_size만큼 100열을 반복해서 가져옴
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

    # 판별자와 생성자의 오차 계산 (d_loss_real과 d_loss_fake의 평균이 바로 오차!)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    g_loss = gan.train_on_batch(noise, true)  # 생성자의 레이블은 무조건 참으로 두고 판별자로 넘김

    print(f'epoch: {i}, d_loss: {round(d_loss, 4)}, g_loss: {round(g_loss, 4)}')


# 2000번 반복되고(+1 하는 거 주의!), 배치 크기는 32, 200번마다 결과가 저장된다
gan_train(2001, 32, 200)

1/1 [==============================] - 0s 236ms/step
epoch: 0, d_loss: 0.6899, g_loss: 0.6651
1/1 [==============================] - 0s 23ms/step
epoch: 1, d_loss: 0.4386, g_loss: 0.2971
1/1 [==============================] - 0s 21ms/step
epoch: 2, d_loss: 0.521, g_loss: 0.071
1/1 [==============================] - 0s 17ms/step
epoch: 3, d_loss: 0.6316, g_loss: 0.0458
1/1 [==============================] - 0s 21ms/step
epoch: 4, d_loss: 0.6129, g_loss: 0.1068
1/1 [==============================] - 0s 26ms/step
epoch: 5, d_loss: 0.4966, g_loss: 0.366
1/1 [==============================] - 0s 21ms/step
epoch: 6, d_loss: 0.4671, g_loss: 0.6675
1/1 [==============================] - 0s 21ms/step
epoch: 7, d_loss: 0.512, g_loss: 0.7705
1/1 [==============================] - 0s 24ms/step
epoch: 8, d_loss: 0.5043, g_loss: 0.7231
1/1 [==============================] - 0s 30ms/step
epoch: 9, d_loss: 0.421, g_loss: 0.7592
1/1 [==============================] - 0s 39ms/step
epoch: 10, d_loss: 0.4